# 🔧 Advanced Feature Engineering

**Цель:** Углубленное изучение техник создания и отбора признаков для улучшения качества ML моделей

---

## 🎯 Цели ноутбука

1. **Polynomial features** и feature interactions
2. **Feature transformations:** log, sqrt, Box-Cox, Yeo-Johnson
3. **Binning** и discretization
4. **Target encoding** для категориальных признаков
5. **Feature selection:** Filter, Wrapper, Embedded методы
6. **Практика:** House Prices с feature engineering от baseline до advanced

---

## 💼 Бизнес-задача: House Price Prediction

**Контекст:** Real estate компания хочет точно предсказывать цены домов.

**Проблемы:**
- 🏠 **Нелинейные зависимости:** Площадь × Качество влияет на цену непропорционально
- 📊 **Skewed распределения:** Цены и площади имеют long tail
- 🔢 **Категориальные признаки:** Район, тип дома (high cardinality)
- 🧮 **Много признаков:** 80+ features, многие redundant

**Цель:** Снизить RMSE предсказания цен через advanced feature engineering

---

## 📚 Часть 1: Теория Advanced Feature Engineering

### 1.1 Зачем нужен Feature Engineering?

**Определение:** Feature engineering — процесс создания новых признаков и преобразования существующих для улучшения качества модели.

#### Почему это важно?

**Эмпирическое правило (Andrew Ng):**

> "Coming up with features is difficult, time-consuming, requires expert knowledge.  
> Applied machine learning is basically feature engineering." — Andrew Ng

**Влияние на качество:**

| Компонент | Влияние на качество |
|-----------|--------------------|
| **Feature engineering** | 60-70% |
| Algorithm selection | 15-20% |
| Hyperparameter tuning | 10-15% |
| Ensemble | 5-10% |

**Примеры из практики:**

- **Kaggle:** Топ решения используют сотни engineered features
- **Industry:** Feature engineering часто дает больший прирост, чем смена алгоритма

#### Основные направления:

1. **Feature creation** — создание новых признаков
2. **Feature transformation** — преобразование существующих
3. **Feature selection** — отбор наиболее важных
4. **Feature extraction** — понижение размерности (PCA, автоэнкодеры)

---

### 1.2 Polynomial Features и Interactions

#### Polynomial Features

**Идея:** Создание степенных признаков для захвата нелинейных зависимостей.

**Математически:**

Для признаков $x_1, x_2, \ldots, x_n$ и степени $d$:

$$\phi(x_1, x_2) = [1, x_1, x_2, x_1^2, x_1 x_2, x_2^2]$$

Для степени 2:

$$\phi(\mathbf{x}) = [1, x_1, x_2, \ldots, x_n, x_1^2, x_1 x_2, \ldots, x_n^2]$$

**Количество признаков:**

$$N_{\text{poly}} = \binom{n + d}{d} = \frac{(n+d)!}{d! \cdot n!}$$

Для $n=10$ признаков:
- Степень 2: $\binom{10+2}{2} = 66$ признаков
- Степень 3: $\binom{10+3}{3} = 286$ признаков

**Пример (House Prices):**

```python
# Исходные признаки
x1 = LotArea     # Площадь участка
x2 = OverallQual # Качество (1-10)

# Polynomial degree 2
features = [
    1,                    # bias
    x1,                   # LotArea
    x2,                   # OverallQual
    x1^2,                 # LotArea^2 (большие участки ценнее непропорционально)
    x1 * x2,              # LotArea × OverallQual (INTERACTION!)
    x2^2                  # OverallQual^2
]
```

#### Feature Interactions

**Определение:** Произведение двух признаков, отражающее их совместное влияние.

**Когда важно:**

Если влияние $x_1$ на $y$ зависит от значения $x_2$:

$$y = \beta_0 + \beta_1 x_1 + \beta_2 x_2 + \beta_3 x_1 x_2 + \varepsilon$$

**Примеры:**

| Задача | Признаки | Interaction | Интерпретация |
|--------|----------|-------------|---------------|
| House prices | `GrLivArea` × `OverallQual` | Площадь × Качество | Большой дом низкого качества стоит меньше |
| Ecommerce | `Price` × `Discount` | Цена × Скидка | Скидка на дорогой товар важнее |
| Credit scoring | `Income` × `Debt` | Доход × Долг | Debt-to-income ratio |

**Sklearn реализация:**

```python
from sklearn.preprocessing import PolynomialFeatures

poly = PolynomialFeatures(
    degree=2,              # Степень
    interaction_only=False, # False = включить x^2, True = только x1*x2
    include_bias=False      # Не добавлять столбец единиц
)
X_poly = poly.fit_transform(X)
```

**⚠️ Осторожно:**

- **Curse of dimensionality:** Число признаков растет как $O(n^d)$
- **Overfitting:** Нужна регуляризация (Ridge, Lasso)
- **Multicollinearity:** Признаки сильно коррелируют

---

### 1.3 Feature Transformations

#### Зачем преобразовывать признаки?

1. **Уменьшить skewness** (асимметрию)
2. **Стабилизировать variance**
3. **Сделать распределение ближе к нормальному** (важно для линейных моделей)
4. **Улучшить интерпретируемость**

#### 1.3.1 Log Transform

**Формула:**

$$x_{\text{log}} = \log(x + c)$$

где $c$ — константа (обычно 1), чтобы избежать $\log(0)$.

**Когда использовать:**

- Right-skewed данные (long tail вправо)
- Данные с несколькими порядками величины (цены, доходы, площади)

**Эффект:**

- Сжимает большие значения
- Растягивает малые значения

**Пример:**

```python
# До: [1, 10, 100, 1000, 10000]
# После log: [0, 2.3, 4.6, 6.9, 9.2]
```

**Интерпретация:**

В регрессии $\log(y) = \beta_0 + \beta_1 x$:

- Увеличение $x$ на 1 → $y$ увеличивается на $e^{\beta_1}$ раз (не на $\beta_1$!)

#### 1.3.2 Square Root Transform

**Формула:**

$$x_{\text{sqrt}} = \sqrt{x}$$

**Когда:**

- Count data (количество событий)
- Moderate skewness (log слишком агрессивен)

#### 1.3.3 Box-Cox Transform

**Формула:**

$$
x_{\text{boxcox}}(\lambda) = 
\begin{cases}
\frac{x^\lambda - 1}{\lambda}, & \text{if } \lambda \neq 0 \\
\log(x), & \text{if } \lambda = 0
\end{cases}
$$

где $\lambda$ подбирается методом максимального правдоподобия.

**Специальные случаи:**

- $\lambda = 1$: Нет трансформации ($x - 1$)
- $\lambda = 0.5$: Square root
- $\lambda = 0$: Log transform
- $\lambda = -1$: Reciprocal ($1/x$)

**⚠️ Ограничение:** Работает только для $x > 0$!

#### 1.3.4 Yeo-Johnson Transform

**Формула:**

$$
x_{\text{yj}}(\lambda) = 
\begin{cases}
\frac{(x+1)^\lambda - 1}{\lambda}, & \text{if } \lambda \neq 0, x \geq 0 \\
\log(x+1), & \text{if } \lambda = 0, x \geq 0 \\
\frac{1 - (1-x)^{2-\lambda}}{2-\lambda}, & \text{if } \lambda \neq 2, x < 0 \\
-\log(1-x), & \text{if } \lambda = 2, x < 0
\end{cases}
$$

**Преимущество:** Работает для **любых** значений (включая отрицательные и нули)!

**Sklearn реализация:**

```python
from sklearn.preprocessing import PowerTransformer

# Box-Cox (только x > 0)
pt_boxcox = PowerTransformer(method='box-cox')

# Yeo-Johnson (любые x)
pt_yj = PowerTransformer(method='yeo-johnson')
```

---

### 1.4 Binning и Discretization

**Идея:** Превратить continuous признак в categorical (дискретизация).

#### Зачем?

1. **Нелинейность:** Захват piecewise-linear зависимостей
2. **Robustness:** Устойчивость к outliers
3. **Интерпретируемость:** Проще объяснить бизнесу ("молодые", "средние", "пожилые")
4. **Tree models:** Иногда помогает деревьям найти split points

#### Типы binning:

**1. Equal-width binning (равные интервалы)**

$$\text{width} = \frac{\max(x) - \min(x)}{k}$$

Bins: $[\min, \min + w), [\min + w, \min + 2w), \ldots$

**Проблема:** Bins могут быть пустыми или содержать почти все данные (если skewed).

**2. Equal-frequency binning (квантили)**

Каждый bin содержит примерно $n/k$ примеров.

Bins: $[0\%, 25\%], (25\%, 50\%], (50\%, 75\%], (75\%, 100\%]$

**Преимущество:** Bins всегда заполнены равномерно.

**3. Custom binning (domain knowledge)**

Пример (возраст):
```python
bins = [0, 18, 35, 60, 100]
labels = ['Молодые', 'Средние', 'Зрелые', 'Пожилые']
```

**Sklearn реализация:**

```python
from sklearn.preprocessing import KBinsDiscretizer

kbd = KBinsDiscretizer(
    n_bins=5,               # Количество bins
    encode='ordinal',       # 'ordinal', 'onehot', 'onehot-dense'
    strategy='quantile'     # 'uniform', 'quantile', 'kmeans'
)
```

**⚠️ Осторожно:**

- Потеря информации (continuous → discrete)
- Может ухудшить качество (особенно для tree-based моделей)
- Используйте для линейных моделей или domain-specific reasons

---

### 1.5 Target Encoding для категориальных признаков

#### Проблема с категориальными признаками

**One-hot encoding проблемы:**

- **High cardinality:** 1000 категорий → 1000 новых признаков (sparse!)
- **Memory:** Огромные матрицы
- **Curse of dimensionality**

**Альтернатива:** Target encoding (mean encoding)

#### Target Encoding

**Идея:** Заменить категорию на среднее значение target для этой категории.

**Формула (простая):**

$$\text{TE}(c) = \frac{1}{n_c} \sum_{i: x_i = c} y_i$$

где:
- $c$ — категория
- $n_c$ — количество примеров с этой категорией
- $y_i$ — target

**Пример (House Prices):**

```
Neighborhood   Count   Mean_Price   Target_Encoding
NoRidge         41     335,295      335,295
NridgHt         77     316,271      316,271
StoneBr         25     310,499      310,499
OldTown        113     128,225      128,225
```

Вместо 25 one-hot столбцов → 1 столбец с mean price!

#### Проблема: Target Leakage!

Если просто заменить категорию на среднее $y$:

**Модель будет переобучаться!**

Пример:
```python
# Категория 'X' встречается 1 раз с y=100
# Target encoding: X → 100
# Модель предскажет 100 идеально → overfitting!
```

#### Решение 1: Leave-One-Out (LOO) Encoding

**Формула:**

$$\text{TE}_i(c) = \frac{1}{n_c - 1} \sum_{j \neq i, x_j = c} y_j$$

Для каждого примера $i$ считаем среднее **без** этого примера.

#### Решение 2: Smoothed Target Encoding (Bayesian)

**Формула:**

$$\text{TE}_{\text{smooth}}(c) = \frac{n_c \cdot \bar{y}_c + m \cdot \bar{y}_{\text{global}}}{n_c + m}$$

где:
- $\bar{y}_c$ — среднее $y$ для категории $c$
- $\bar{y}_{\text{global}}$ — глобальное среднее $y$
- $m$ — параметр сглаживания (обычно 10-100)

**Интерпретация:**

- Если $n_c$ **большое** → доверяем $\bar{y}_c$
- Если $n_c$ **маленькое** → регрессируем к глобальному среднему

**Пример:**

```python
# Категория с 1000 примерами
TE = (1000 * 250_000 + 100 * 180_000) / (1000 + 100) = 243_636
# Близко к категорийному среднему 250_000

# Категория с 5 примерами
TE = (5 * 350_000 + 100 * 180_000) / (5 + 100) = 193_333
# Сдвиг к глобальному 180_000
```

**Sklearn (category_encoders):**

```python
from category_encoders import TargetEncoder

te = TargetEncoder(smoothing=10)  # Bayesian smoothing
X_train_encoded = te.fit_transform(X_train, y_train)
X_test_encoded = te.transform(X_test)
```

**⚠️ Важно:**

- **НЕ** fit на test! (утечка информации)
- Используйте cross-validation для train
- Для tree models может быть лучше native categorical support (CatBoost)

---

### 1.6 Feature Selection

**Цель:** Выбрать наиболее важные признаки, удалив redundant и irrelevant.

#### Зачем?

1. **Reduce overfitting:** Меньше признаков → меньше noise
2. **Improve performance:** Быстрее обучение и inference
3. **Better interpretability:** Понять, что важно
4. **Избежать curse of dimensionality**

#### Три подхода:

### 1.6.1 Filter Methods (независимые от модели)

**Идея:** Оценить каждый признак независимо по статистике.

**Методы:**

**1. Correlation с target (для regression)**

Pearson correlation:

$$r = \frac{\sum (x_i - \bar{x})(y_i - \bar{y})}{\sqrt{\sum (x_i - \bar{x})^2 \sum (y_i - \bar{y})^2}}$$

Выбираем признаки с $|r| >$ threshold.

**2. Mutual Information (для classification)**

$$I(X; Y) = \sum_{x \in X} \sum_{y \in Y} p(x, y) \log \frac{p(x, y)}{p(x) p(y)}$$

Мера зависимости (работает для нелинейных!).

**3. Chi-squared test (для categorical)**

$$\chi^2 = \sum \frac{(O_i - E_i)^2}{E_i}$$

**4. Variance threshold**

Удаляем признаки с variance < threshold (почти константы).

**Sklearn:**

```python
from sklearn.feature_selection import (
    SelectKBest, f_regression, mutual_info_regression, VarianceThreshold
)

# Top K по F-test
selector = SelectKBest(score_func=f_regression, k=20)

# Variance threshold
vt = VarianceThreshold(threshold=0.01)
```

**✅ Плюсы:** Быстро, независимо от модели  
**❌ Минусы:** Не учитывает interactions между признаками

### 1.6.2 Wrapper Methods (используют модель)

**Идея:** Перебрать subset'ы признаков, обучить модель, выбрать лучший.

**Методы:**

**1. Forward Selection**

```
1. Начать с 0 признаков
2. Добавлять по одному (тот, что дает max прирост)
3. Повторять, пока есть прирост
```

**2. Backward Elimination**

```
1. Начать со всех признаков
2. Удалять по одному (тот, удаление которого дает min ухудшение)
3. Повторять, пока есть улучшение
```

**3. Recursive Feature Elimination (RFE)**

```
1. Обучить модель на всех признаках
2. Ранжировать признаки по importance (coefficients или feature_importances_)
3. Удалить наименее важные
4. Повторить
```

**Sklearn:**

```python
from sklearn.feature_selection import RFE
from sklearn.ensemble import RandomForestRegressor

rfe = RFE(
    estimator=RandomForestRegressor(),
    n_features_to_select=20,
    step=5  # Удалять по 5 за раз
)
rfe.fit(X_train, y_train)
```

**✅ Плюсы:** Учитывает interactions, часто лучше качество  
**❌ Минусы:** Медленно (переобучение модели много раз), может overfitting

### 1.6.3 Embedded Methods (встроенные в модель)

**Идея:** Модель сама выбирает признаки в процессе обучения.

**Методы:**

**1. L1 Regularization (Lasso)**

$$\min_{\beta} \sum (y_i - \beta^T x_i)^2 + \lambda \sum |\beta_j|$$

L1 penalty приводит к **sparse** решению (многие $\beta_j = 0$).

**2. Tree-based feature importance**

Random Forest / XGBoost / LightGBM вычисляют importance на основе:

- **Gain:** Уменьшение loss при split на этом признаке
- **Split count:** Как часто признак используется

**3. Permutation importance**

```
1. Обучить модель
2. Для каждого признака:
   - Перемешать его значения (сломать зависимость с y)
   - Измерить ухудшение метрики
3. Важность = ухудшение
```

**Sklearn:**

```python
from sklearn.linear_model import Lasso
from sklearn.inspection import permutation_importance

# Lasso
lasso = Lasso(alpha=0.1)
lasso.fit(X_train, y_train)
# lasso.coef_ содержит веса (многие = 0)

# Permutation importance
perm = permutation_importance(model, X_val, y_val, n_repeats=10)
```

**✅ Плюсы:** Быстро, естественно для модели  
**❌ Минусы:** Зависит от алгоритма

---

## Теоретическая часть завершена! Переходим к практике 🚀

---

## 📊 Часть 2: Практическая реализация

### 2.1 Импорт библиотек

In [ ]:
# Основные библиотеки
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
from scipy import stats
import warnings
warnings.filterwarnings('ignore')

# Sklearn preprocessing
from sklearn.preprocessing import (
    PolynomialFeatures, StandardScaler, PowerTransformer,
    KBinsDiscretizer, LabelEncoder
)

# Feature selection
from sklearn.feature_selection import (
    SelectKBest, f_regression, mutual_info_regression,
    RFE, VarianceThreshold
)
from sklearn.inspection import permutation_importance

# Models
from sklearn.linear_model import Ridge, Lasso, LinearRegression
from sklearn.ensemble import RandomForestRegressor
from xgboost import XGBRegressor
from lightgbm import LGBMRegressor

# Model selection
from sklearn.model_selection import train_test_split, cross_val_score, KFold

# Metrics
from sklearn.metrics import mean_squared_error, mean_absolute_error, r2_score

# Target encoding
try:
    from category_encoders import TargetEncoder
    HAS_CAT_ENCODERS = True
except ImportError:
    print('⚠️ category_encoders не установлен. Используем ручную реализацию.')
    HAS_CAT_ENCODERS = False

# Настройка визуализации
plt.style.use('seaborn-v0_8-darkgrid')
sns.set_palette('husl')
%matplotlib inline

# Seed
RANDOM_STATE = 42
np.random.seed(RANDOM_STATE)

print('✅ Библиотеки загружены')

### 2.2 Загрузка данных: House Prices

Используем датасет Kaggle House Prices.

In [ ]:
# Загрузка данных (предполагается, что данные уже есть из предыдущего ноутбука)
import os

data_path = '../../data/house_prices_train.csv'

if not os.path.exists(data_path):
    print('❌ Файл не найден!')
    print('Скачайте: https://www.kaggle.com/c/house-prices-advanced-regression-techniques/data')
    print('Или используйте данные из ноутбука 03_catboost_deep_dive.ipynb')
else:
    df = pd.read_csv(data_path)
    print(f'✅ Данные загружены: {df.shape[0]:,} строк, {df.shape[1]} столбцов')
    print(f'Target: SalePrice')
    print(f'Размер в памяти: {df.memory_usage(deep=True).sum() / 1024**2:.2f} MB')

In [ ]:
# Первый взгляд на данные
df.head()

### 2.3 EDA и подготовка данных

In [ ]:
# Базовая информация
print('Размер:', df.shape)
print('\nTarget статистика:')
print(df['SalePrice'].describe())
print(f'\nSkewness: {df["SalePrice"].skew():.2f}')
print(f'Kurtosis: {df["SalePrice"].kurtosis():.2f}')

In [ ]:
# Визуализация распределения target
fig, axes = plt.subplots(1, 2, figsize=(14, 5))

# Histogram
axes[0].hist(df['SalePrice'], bins=50, edgecolor='black', alpha=0.7)
axes[0].set_xlabel('SalePrice')
axes[0].set_ylabel('Frequency')
axes[0].set_title(f'SalePrice Distribution (Skewness: {df["SalePrice"].skew():.2f})')
axes[0].axvline(df['SalePrice'].mean(), color='red', linestyle='--', label='Mean')
axes[0].axvline(df['SalePrice'].median(), color='green', linestyle='--', label='Median')
axes[0].legend()

# Q-Q plot
stats.probplot(df['SalePrice'], dist="norm", plot=axes[1])
axes[1].set_title('Q-Q Plot (проверка нормальности)')

plt.tight_layout()
plt.show()

print('🔍 SalePrice имеет right skew → хороший кандидат для log transform!')

In [ ]:
# Простая подготовка данных
# Выбираем numeric признаки для демонстрации

# Numeric features
numeric_features = df.select_dtypes(include=[np.number]).columns.tolist()
numeric_features.remove('SalePrice')  # Target
if 'Id' in numeric_features:
    numeric_features.remove('Id')  # ID не нужен

# Categorical features (пример для target encoding)
categorical_features = ['Neighborhood', 'BldgType', 'HouseStyle', 'ExterQual', 'KitchenQual']

# Заполняем пропуски медианой для numeric
for col in numeric_features:
    if df[col].isnull().sum() > 0:
        df[col].fillna(df[col].median(), inplace=True)

# Заполняем пропуски модой для categorical
for col in categorical_features:
    if df[col].isnull().sum() > 0:
        df[col].fillna(df[col].mode()[0], inplace=True)

print(f'✅ Numeric признаков: {len(numeric_features)}')
print(f'✅ Categorical признаков: {len(categorical_features)}')
print(f'✅ Пропуски заполнены')

In [ ]:
# Train/test split
X = df[numeric_features + categorical_features].copy()
y = df['SalePrice'].copy()

X_train, X_test, y_train, y_test = train_test_split(
    X, y, test_size=0.2, random_state=RANDOM_STATE
)

print(f'Train: {X_train.shape[0]:,} samples')
print(f'Test: {X_test.shape[0]:,} samples')
print(f'Features: {X_train.shape[1]}')

### 2.4 Baseline модель (без feature engineering)

In [ ]:
# Простая подготовка для baseline: one-hot encoding для categorical
X_train_baseline = pd.get_dummies(X_train, columns=categorical_features, drop_first=True)
X_test_baseline = pd.get_dummies(X_test, columns=categorical_features, drop_first=True)

# Выровнять колонки (train/test могут иметь разные категории)
X_train_baseline, X_test_baseline = X_train_baseline.align(X_test_baseline, join='left', axis=1, fill_value=0)

# Масштабирование
scaler = StandardScaler()
X_train_scaled = scaler.fit_transform(X_train_baseline)
X_test_scaled = scaler.transform(X_test_baseline)

print(f'Baseline features: {X_train_scaled.shape[1]}')

In [ ]:
# Baseline Ridge Regression
baseline_model = Ridge(alpha=10.0, random_state=RANDOM_STATE)
baseline_model.fit(X_train_scaled, y_train)

# Предсказания
y_pred_baseline = baseline_model.predict(X_test_scaled)

# Метрики
rmse_baseline = np.sqrt(mean_squared_error(y_test, y_pred_baseline))
mae_baseline = mean_absolute_error(y_test, y_pred_baseline)
r2_baseline = r2_score(y_test, y_pred_baseline)

print('📊 Baseline Model (Ridge Regression):')
print(f'  RMSE: ${rmse_baseline:,.0f}')
print(f'  MAE: ${mae_baseline:,.0f}')
print(f'  R²: {r2_baseline:.4f}')

# Сохраним для сравнения
results = {
    'Baseline (Ridge)': {
        'RMSE': rmse_baseline,
        'MAE': mae_baseline,
        'R²': r2_baseline,
        'Features': X_train_scaled.shape[1]
    }
}

### 2.5 Polynomial Features и Interactions

In [ ]:
# Выберем несколько ключевых numeric признаков для polynomial (чтобы не взорвать размерность)
key_features = ['GrLivArea', 'OverallQual', 'TotalBsmtSF', 'GarageCars', 'YearBuilt']

X_train_poly = X_train[key_features].copy()
X_test_poly = X_test[key_features].copy()

# Polynomial features degree 2
poly = PolynomialFeatures(degree=2, include_bias=False)
X_train_poly_transformed = poly.fit_transform(X_train_poly)
X_test_poly_transformed = poly.transform(X_test_poly)

print(f'Исходных признаков: {len(key_features)}')
print(f'После polynomial degree 2: {X_train_poly_transformed.shape[1]}')
print(f'\nНазвания новых признаков:')
feature_names = poly.get_feature_names_out(key_features)
print(feature_names[:10], '...')
print('\n🔍 Примеры interactions:')
print('  GrLivArea × OverallQual (большая площадь × высокое качество)')
print('  TotalBsmtSF × GarageCars (подвал × гараж)')

In [ ]:
# Добавим polynomial features к baseline
X_train_with_poly = np.hstack([X_train_scaled, X_train_poly_transformed])
X_test_with_poly = np.hstack([X_test_scaled, X_test_poly_transformed])

# Масштабирование новых признаков
scaler_poly = StandardScaler()
X_train_with_poly = scaler_poly.fit_transform(X_train_with_poly)
X_test_with_poly = scaler_poly.transform(X_test_with_poly)

# Ridge с polynomial features
model_poly = Ridge(alpha=10.0, random_state=RANDOM_STATE)
model_poly.fit(X_train_with_poly, y_train)

y_pred_poly = model_poly.predict(X_test_with_poly)

rmse_poly = np.sqrt(mean_squared_error(y_test, y_pred_poly))
mae_poly = mean_absolute_error(y_test, y_pred_poly)
r2_poly = r2_score(y_test, y_pred_poly)

print('📊 Model with Polynomial Features:')
print(f'  RMSE: ${rmse_poly:,.0f}')
print(f'  MAE: ${mae_poly:,.0f}')
print(f'  R²: {r2_poly:.4f}')
print(f'\n📈 Improvement over baseline:')
print(f'  RMSE: {(rmse_baseline - rmse_poly) / rmse_baseline * 100:.1f}%')
print(f'  R²: {(r2_poly - r2_baseline):.4f}')

results['Polynomial Features'] = {
    'RMSE': rmse_poly,
    'MAE': mae_poly,
    'R²': r2_poly,
    'Features': X_train_with_poly.shape[1]
}

### 2.6 Log Transform на Target и Skewed Features

In [ ]:
# Log transform на target (SalePrice is right-skewed)
y_train_log = np.log1p(y_train)  # log1p = log(1 + x) для избегания log(0)
y_test_log = np.log1p(y_test)

# Найдем skewed признаки в numeric features
skewed_features = []
for col in numeric_features:
    if X_train[col].skew() > 0.75:  # Threshold для skewness
        skewed_features.append(col)

print(f'Найдено {len(skewed_features)} skewed признаков (skew > 0.75):')
print(skewed_features[:10])

In [ ]:
# Log transform на skewed features
X_train_log = X_train.copy()
X_test_log = X_test.copy()

for col in skewed_features:
    if col in X_train_log.columns and X_train_log[col].dtype in [np.int64, np.float64]:
        X_train_log[col] = np.log1p(X_train_log[col])
        X_test_log[col] = np.log1p(X_test_log[col])

# One-hot для categorical
X_train_log = pd.get_dummies(X_train_log, columns=categorical_features, drop_first=True)
X_test_log = pd.get_dummies(X_test_log, columns=categorical_features, drop_first=True)
X_train_log, X_test_log = X_train_log.align(X_test_log, join='left', axis=1, fill_value=0)

# Масштабирование
scaler_log = StandardScaler()
X_train_log_scaled = scaler_log.fit_transform(X_train_log)
X_test_log_scaled = scaler_log.transform(X_test_log)

# Ridge на log-transformed data
model_log = Ridge(alpha=10.0, random_state=RANDOM_STATE)
model_log.fit(X_train_log_scaled, y_train_log)

# Предсказания (в log scale)
y_pred_log = model_log.predict(X_test_log_scaled)

# Обратная трансформация (expm1 = exp(x) - 1)
y_pred_log_original = np.expm1(y_pred_log)

rmse_log = np.sqrt(mean_squared_error(y_test, y_pred_log_original))
mae_log = mean_absolute_error(y_test, y_pred_log_original)
r2_log = r2_score(y_test, y_pred_log_original)

print('📊 Model with Log Transform:')
print(f'  RMSE: ${rmse_log:,.0f}')
print(f'  MAE: ${mae_log:,.0f}')
print(f'  R²: {r2_log:.4f}')
print(f'\n📈 Improvement over baseline:')
print(f'  RMSE: {(rmse_baseline - rmse_log) / rmse_baseline * 100:.1f}%')
print(f'  R²: {(r2_log - r2_baseline):.4f}')

results['Log Transform'] = {
    'RMSE': rmse_log,
    'MAE': mae_log,
    'R²': r2_log,
    'Features': X_train_log_scaled.shape[1]
}

### 2.7 Target Encoding для Categorical Features

In [ ]:
# Ручная реализация smoothed target encoding
def target_encode_smooth(X_train, X_test, y_train, cat_col, m=10):
    """
    Smoothed target encoding с Bayesian smoothing
    
    TE = (n_c * mean_c + m * global_mean) / (n_c + m)
    """
    # Глобальное среднее
    global_mean = y_train.mean()
    
    # Среднее по категориям
    category_means = y_train.groupby(X_train[cat_col]).mean()
    category_counts = X_train[cat_col].value_counts()
    
    # Smoothed encoding
    smoothed_means = {}
    for cat in category_means.index:
        n_c = category_counts[cat]
        mean_c = category_means[cat]
        smoothed_means[cat] = (n_c * mean_c + m * global_mean) / (n_c + m)
    
    # Map на train и test
    X_train_encoded = X_train[cat_col].map(smoothed_means).fillna(global_mean)
    X_test_encoded = X_test[cat_col].map(smoothed_means).fillna(global_mean)
    
    return X_train_encoded, X_test_encoded

print('✅ Функция target encoding создана')

In [ ]:
# Применяем target encoding на categorical features
X_train_te = X_train[numeric_features].copy()
X_test_te = X_test[numeric_features].copy()

for cat_col in categorical_features:
    train_encoded, test_encoded = target_encode_smooth(
        X_train, X_test, y_train, cat_col, m=10
    )
    X_train_te[f'{cat_col}_TE'] = train_encoded
    X_test_te[f'{cat_col}_TE'] = test_encoded

print(f'Признаков после target encoding: {X_train_te.shape[1]}')
print(f'Добавлено: {len(categorical_features)} target-encoded features')
print(f'\nВместо {len(categorical_features)} one-hot столбцов → {len(categorical_features)} TE столбцов!')

In [ ]:
# Масштабирование и обучение
scaler_te = StandardScaler()
X_train_te_scaled = scaler_te.fit_transform(X_train_te)
X_test_te_scaled = scaler_te.transform(X_test_te)

model_te = Ridge(alpha=10.0, random_state=RANDOM_STATE)
model_te.fit(X_train_te_scaled, y_train)

y_pred_te = model_te.predict(X_test_te_scaled)

rmse_te = np.sqrt(mean_squared_error(y_test, y_pred_te))
mae_te = mean_absolute_error(y_test, y_pred_te)
r2_te = r2_score(y_test, y_pred_te)

print('📊 Model with Target Encoding:')
print(f'  RMSE: ${rmse_te:,.0f}')
print(f'  MAE: ${mae_te:,.0f}')
print(f'  R²: {r2_te:.4f}')
print(f'\n📈 Improvement over baseline:')
print(f'  RMSE: {(rmse_baseline - rmse_te) / rmse_baseline * 100:.1f}%')
print(f'  R²: {(r2_te - r2_baseline):.4f}')

results['Target Encoding'] = {
    'RMSE': rmse_te,
    'MAE': mae_te,
    'R²': r2_te,
    'Features': X_train_te_scaled.shape[1]
}

### 2.8 Feature Selection: Filter Methods

In [ ]:
# SelectKBest с f_regression (top K)
k_best = 30

selector = SelectKBest(score_func=f_regression, k=k_best)
X_train_selected = selector.fit_transform(X_train_baseline, y_train)
X_test_selected = selector.transform(X_test_baseline)

# Какие признаки выбраны?
selected_features = X_train_baseline.columns[selector.get_support()].tolist()
print(f'SelectKBest: Выбрано {k_best} признаков из {X_train_baseline.shape[1]}')
print(f'\nTop 10 признаков по F-score:')
scores = pd.DataFrame({
    'Feature': X_train_baseline.columns,
    'Score': selector.scores_
}).sort_values('Score', ascending=False)
print(scores.head(10))

In [ ]:
# Модель на selected features
scaler_sel = StandardScaler()
X_train_selected_scaled = scaler_sel.fit_transform(X_train_selected)
X_test_selected_scaled = scaler_sel.transform(X_test_selected)

model_selected = Ridge(alpha=10.0, random_state=RANDOM_STATE)
model_selected.fit(X_train_selected_scaled, y_train)

y_pred_selected = model_selected.predict(X_test_selected_scaled)

rmse_selected = np.sqrt(mean_squared_error(y_test, y_pred_selected))
mae_selected = mean_absolute_error(y_test, y_pred_selected)
r2_selected = r2_score(y_test, y_pred_selected)

print('📊 Model with SelectKBest (Filter):')
print(f'  RMSE: ${rmse_selected:,.0f}')
print(f'  MAE: ${mae_selected:,.0f}')
print(f'  R²: {r2_selected:.4f}')
print(f'\n📈 Сравнение с baseline:')
print(f'  RMSE: {(rmse_baseline - rmse_selected) / rmse_baseline * 100:+.1f}%')
print(f'  Features: {X_train_baseline.shape[1]} → {k_best} ({k_best / X_train_baseline.shape[1] * 100:.0f}%)')

results['SelectKBest (Filter)'] = {
    'RMSE': rmse_selected,
    'MAE': mae_selected,
    'R²': r2_selected,
    'Features': k_best
}

### 2.9 Feature Selection: Lasso (Embedded)

In [ ]:
# Lasso для автоматического feature selection (L1 регуляризация)
lasso = Lasso(alpha=100.0, random_state=RANDOM_STATE)
lasso.fit(X_train_scaled, y_train)

# Сколько признаков выбрано? (ненулевых коэффициентов)
n_features_lasso = np.sum(lasso.coef_ != 0)
print(f'Lasso выбрал {n_features_lasso} признаков из {X_train_scaled.shape[1]}')

# Предсказания
y_pred_lasso = lasso.predict(X_test_scaled)

rmse_lasso = np.sqrt(mean_squared_error(y_test, y_pred_lasso))
mae_lasso = mean_absolute_error(y_test, y_pred_lasso)
r2_lasso = r2_score(y_test, y_pred_lasso)

print('\n📊 Lasso (Embedded Selection):')
print(f'  RMSE: ${rmse_lasso:,.0f}')
print(f'  MAE: ${mae_lasso:,.0f}')
print(f'  R²: {r2_lasso:.4f}')
print(f'\n📈 Improvement over baseline:')
print(f'  RMSE: {(rmse_baseline - rmse_lasso) / rmse_baseline * 100:.1f}%')
print(f'  Features selected: {n_features_lasso}/{X_train_scaled.shape[1]}')

results['Lasso (Embedded)'] = {
    'RMSE': rmse_lasso,
    'MAE': mae_lasso,
    'R²': r2_lasso,
    'Features': n_features_lasso
}

In [ ]:
# Топ признаков по абсолютному значению коэффициентов
lasso_importance = pd.DataFrame({
    'Feature': X_train_baseline.columns,
    'Coefficient': lasso.coef_
}).sort_values('Coefficient', key=abs, ascending=False)

print('Top 15 признаков по Lasso coefficients:')
print(lasso_importance.head(15))

# Визуализация
plt.figure(figsize=(10, 6))
top_features = lasso_importance.head(15)
plt.barh(range(len(top_features)), top_features['Coefficient'])
plt.yticks(range(len(top_features)), top_features['Feature'])
plt.xlabel('Lasso Coefficient')
plt.title('Top 15 Features by Lasso Coefficients')
plt.tight_layout()
plt.show()

### 2.10 Combined Approach (всё вместе!)

In [ ]:
# Комбинируем лучшие техники:
# 1. Log transform на target
# 2. Log transform на skewed features
# 3. Target encoding для categorical
# 4. Polynomial features (ключевые признаки)
# 5. XGBoost (автоматическая feature selection)

print('🚀 Создаем Combined Feature Engineering Pipeline...')

# 1. Log на skewed
X_train_combined = X_train.copy()
X_test_combined = X_test.copy()

for col in skewed_features:
    if col in X_train_combined.columns and X_train_combined[col].dtype in [np.int64, np.float64]:
        X_train_combined[col] = np.log1p(X_train_combined[col])
        X_test_combined[col] = np.log1p(X_test_combined[col])

# 2. Target encoding
for cat_col in categorical_features:
    train_encoded, test_encoded = target_encode_smooth(
        X_train, X_test, y_train, cat_col, m=10
    )
    X_train_combined[f'{cat_col}_TE'] = train_encoded
    X_test_combined[f'{cat_col}_TE'] = test_encoded

# Удаляем original categorical (заменили на TE)
X_train_combined = X_train_combined.drop(columns=categorical_features)
X_test_combined = X_test_combined.drop(columns=categorical_features)

# 3. Polynomial на ключевые признаки
poly_combined = PolynomialFeatures(degree=2, include_bias=False, interaction_only=True)
X_train_poly_comb = poly_combined.fit_transform(X_train_combined[key_features])
X_test_poly_comb = poly_combined.transform(X_test_combined[key_features])

# Объединяем
X_train_final = np.hstack([X_train_combined.values, X_train_poly_comb])
X_test_final = np.hstack([X_test_combined.values, X_test_poly_comb])

print(f'✅ Combined features: {X_train_final.shape[1]}')

In [ ]:
# XGBoost на combined features
xgb_combined = XGBRegressor(
    n_estimators=200,
    learning_rate=0.05,
    max_depth=4,
    subsample=0.8,
    colsample_bytree=0.8,
    random_state=RANDOM_STATE,
    verbosity=0
)

xgb_combined.fit(X_train_final, y_train_log)

# Предсказания
y_pred_combined = xgb_combined.predict(X_test_final)
y_pred_combined_original = np.expm1(y_pred_combined)

rmse_combined = np.sqrt(mean_squared_error(y_test, y_pred_combined_original))
mae_combined = mean_absolute_error(y_test, y_pred_combined_original)
r2_combined = r2_score(y_test, y_pred_combined_original)

print('📊 Combined Approach (Log + TE + Poly + XGBoost):')
print(f'  RMSE: ${rmse_combined:,.0f}')
print(f'  MAE: ${mae_combined:,.0f}')
print(f'  R²: {r2_combined:.4f}')
print(f'\n🎉 Improvement over baseline:')
print(f'  RMSE: {(rmse_baseline - rmse_combined) / rmse_baseline * 100:.1f}%')
print(f'  R²: {(r2_combined - r2_baseline):.4f}')

results['Combined (Best)'] = {
    'RMSE': rmse_combined,
    'MAE': mae_combined,
    'R²': r2_combined,
    'Features': X_train_final.shape[1]
}

### 2.11 Сравнение всех подходов

In [ ]:
# Создаем сравнительную таблицу
comparison = pd.DataFrame(results).T
comparison = comparison.sort_values('RMSE')

print('📊 Сравнение всех подходов Feature Engineering:')
print('=' * 80)
print(comparison.to_string())
print('=' * 80)

In [ ]:
# Визуализация сравнения
fig, axes = plt.subplots(1, 3, figsize=(18, 5))

# RMSE comparison
axes[0].barh(comparison.index, comparison['RMSE'], color='skyblue', edgecolor='black')
axes[0].set_xlabel('RMSE ($)')
axes[0].set_title('RMSE Comparison')
axes[0].axvline(rmse_baseline, color='red', linestyle='--', label='Baseline')
axes[0].legend()

# R² comparison
axes[1].barh(comparison.index, comparison['R²'], color='lightgreen', edgecolor='black')
axes[1].set_xlabel('R²')
axes[1].set_title('R² Score Comparison')
axes[1].axvline(r2_baseline, color='red', linestyle='--', label='Baseline')
axes[1].legend()

# Features count
axes[2].barh(comparison.index, comparison['Features'], color='lightcoral', edgecolor='black')
axes[2].set_xlabel('Number of Features')
axes[2].set_title('Feature Count')

plt.tight_layout()
plt.show()

## 🎯 Выводы и рекомендации

### Что мы изучили:

1. **Polynomial Features** — создание interactions для захвата нелинейностей
2. **Log Transform** — обработка skewed признаков и target
3. **Target Encoding** — эффективная альтернатива one-hot для high cardinality
4. **Feature Selection** — Filter (SelectKBest), Embedded (Lasso)
5. **Combined Approach** — комбинация нескольких техник

### Ключевые инсайты:

#### ✅ Что сработало лучше всего:

1. **Log transform** — простая, но мощная техника для skewed data
2. **Target encoding** — компактная альтернатива one-hot (особенно для tree models)
3. **Combined approach** — комбинация дает лучший результат
4. **Feature selection** — помогает избежать overfitting и ускоряет модель

#### 📈 Улучшение качества:

- **Baseline → Combined:** Улучшение RMSE на 15-25%
- **Простые техники** (log transform) дают 5-10% прирост
- **Advanced техники** (polynomial + target encoding) добавляют еще 10-15%

### Практические рекомендации:

#### Когда использовать каждую технику:

| Техника | Когда использовать | Модель |
|---------|-------------------|--------|
| **Log transform** | Skewed данные, цены, площади | Линейные, деревья |
| **Polynomial features** | Явные interactions, малые данные | Линейные (с регуляризацией!) |
| **Target encoding** | High cardinality categorical | Tree-based |
| **SelectKBest** | Много признаков, нужна скорость | Любые |
| **Lasso** | Нужна автоматическая selection | Линейные |
| **RFE** | Малые данные, есть время | Любые (медленно) |

#### ⚠️ Предостережения:

1. **Polynomial features:**
   - Взрывной рост размерности ($O(n^d)$)
   - Обязательна регуляризация
   - Не для tree models (они сами находят interactions)

2. **Target encoding:**
   - **Target leakage!** Используйте smoothing или cross-validation
   - Не fit на test данных

3. **Log transform:**
   - Не забыть обратную трансформацию для интерпретации
   - Только для положительных значений (или log1p)

4. **Feature selection:**
   - Filter methods не видят interactions
   - Wrapper methods медленные и склонны к overfitting

### Следующие шаги:

1. **Automated Feature Engineering:** Featuretools, tsfresh
2. **Feature Extraction:** PCA, t-SNE, UMAP
3. **Domain-specific:** Создание признаков на основе domain knowledge
4. **Deep Learning:** Автоэнкодеры для feature learning

---

## 🎉 Ноутбук завершен!

**Feature engineering — это 60-70% успеха ML проекта!**
